# ChatBot 

In [25]:
import os 
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")


In [26]:
from langchain_groq import ChatGroq
llm = ChatGroq(model = "gemma2-9b-it")


In [27]:
# incorporating Message History

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

store = {}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


In [28]:
# incorporating prompt template

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer the questions in {language}"),
        MessagesPlaceholder(variable_name = "messages"),
    ]
)


In [36]:
# managing Conversation History

from langchain_core.messages import trim_messages
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

trimmer = trim_messages(
    max_tokens = 70,
    strategy = "last",
    token_counter = llm,
    include_system = True,
    allow_partial = False,
    start_on = "human"
)


In [37]:
# final chain and invokable 

chain = (
    RunnablePassthrough.assign(messages = itemgetter("messages")|trimmer)
    | prompt
    | llm
)

with_message_history = RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key = "messages"
)

config0 = {"configurable":{"session_id":"Chat0"}}

In [39]:
messages = [
    SystemMessage(content= "You're a good assistant"),
    HumanMessage(content= "Hi! I am Rob"),
    AIMessage(content= "Hi!"),
    HumanMessage(content= "I like Chocolate Ice cream"),
    AIMessage(content= "nice"),
    HumanMessage(content= "what's 2+2"),
    AIMessage(content= "4"),
    HumanMessage(content= "Thanks!"),
    AIMessage(content= "no problem"),
    HumanMessage(content= "Having fun?"),
    AIMessage(content= "yes"),
]

response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content= "What is the last mathematic question i asked you?")],
        "language": "English",
    },
    config = config0,
)

response

AIMessage(content='The last math question you asked me was "what\'s 2+2". 😊  \n\n\n\nI\'m ready for another one if you have it!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 117, 'total_tokens': 152, 'completion_time': 0.063636364, 'prompt_time': 0.005099827, 'queue_time': 0.259787422, 'total_time': 0.068736191}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--89562ee2-7f2c-496f-8a90-5be231927cea-0', usage_metadata={'input_tokens': 117, 'output_tokens': 35, 'total_tokens': 152})